In [ ]:
'''
------------------------------------------------
Análisis de modelos predictivos en bolsa
Copyright (C) 2024-2025 MegaStorm Systems

This software is provided 'as-is', without any express or implied
warranty.  In no event will the authors be held liable for any damages
arising from the use of this software.

Permission is granted to anyone to use this software for any purpose,
including commercial applications, and to alter it and redistribute it
freely, subject to the following restrictions:

1. The origin of this software must not be misrepresented; you must not
claim that you wrote the original software. If you use this software
in a product, an acknowledgment in the product documentation would be
appreciated but is not required.
2. Altered source versions must be plainly marked as such, and must not be
misrepresented as being the original software.
3. This notice may not be removed or altered from any source distribution.

------------------------------------------------------------------------------------------------
Orquestador: ejecuta de forma automatizada todos los modelos con sus variantes v3.1
------------------------------------------------------------------------------------------------'''

In [ ]:
# Importar librerias
import subprocess
import itertools
import datetime
import time
import threading
import os
import sys
import pandas as pd
import glob
import plotly.graph_objects as go
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor, as_completed
from ampblib import AMPBConfig

In [ ]:
# Verifica si los archivos de salida ya existen para evitar re-ejecutar.
# Retorna True si TODOS los archivos existen, False si falta alguno.
def checkOutputFiles(model_title, params):
    
    files_to_check = []
    
    if model_title == "SARIMAX":
        # Para SARIMAX: 4 archivos (SV y BT, cada uno .png y .csv)
        base_name = f"SARIMAX_SV_{params['transformation']}_{params['exog_scaling']}_{params['exog_set_id']}"
        files_to_check.extend([
            f"{AMPBConfig.OUTPUT_DIR}/SARIMAX/SV/{base_name}.png",
            f"{AMPBConfig.OUTPUT_DIR}/SARIMAX/SV/{base_name}.csv"
        ])
        
        base_name_bt = f"SARIMAX_BT_{params['transformation']}_{params['exog_scaling']}_{params['exog_set_id']}"
        files_to_check.extend([
            f"{AMPBConfig.OUTPUT_DIR}/SARIMAX/BT/{base_name_bt}.png",
            f"{AMPBConfig.OUTPUT_DIR}/SARIMAX/BT/{base_name_bt}.csv"
        ])
        
    elif model_title == "RF" or model_title == "RandomForest":
        # Para RandomForest: 4 archivos (SV y BT, cada uno .png y .csv)
        base_name_sv = f"RandomForest_SV_{params['exog_set_id']}"
        files_to_check.extend([
            f"{AMPBConfig.OUTPUT_DIR}/RandomForest/SV/{base_name_sv}.png",
            f"{AMPBConfig.OUTPUT_DIR}/RandomForest/SV/{base_name_sv}.csv"
        ])
        
        base_name_bt = f"RandomForest_BT_{params['exog_set_id']}"
        files_to_check.extend([
            f"{AMPBConfig.OUTPUT_DIR}/RandomForest/BT/{base_name_bt}.png",
            f"{AMPBConfig.OUTPUT_DIR}/RandomForest/BT/{base_name_bt}.csv"
        ])

    elif model_title == "XGBoost":
        # Para XGBoost: 4 archivos (SV y BT, cada uno .png y .csv)
        base_name_sv = f"XGBoost_SV_{params['exog_set_id']}"
        files_to_check.extend([
            f"{AMPBConfig.OUTPUT_DIR}/XGBoost/SV/{base_name_sv}.png",
            f"{AMPBConfig.OUTPUT_DIR}/XGBoost/SV/{base_name_sv}.csv"
        ])
        
        base_name_bt = f"XGBoost_BT_{params['exog_set_id']}"
        files_to_check.extend([
            f"{AMPBConfig.OUTPUT_DIR}/XGBoost/BT/{base_name_bt}.png",
            f"{AMPBConfig.OUTPUT_DIR}/XGBoost/BT/{base_name_bt}.csv"
        ])
    
    else:
        # Para otros modelos, no verificar archivos (ejecutar siempre)
        return False
    
    # Verificar si TODOS los archivos existen
    all_exist = all(os.path.exists(file_path) for file_path in files_to_check)
    
    if all_exist:
        print(f"  {AMPBConfig.COLOR_KEY}Saltando ejecución: archivos de resultados ya existen:{AMPBConfig.COLOR_RESET}")
        for file_path in files_to_check:
            print(f"    ✓ {file_path}")
    print("")

    return all_exist

In [ ]:
# Comprueba que los resultados no existan. Permite ejecucion incremental.
def runGridSearch(model_title, worker_notebook, param_grid, debug_mode=False):
    # Crear directorio para el modelo y archivo de log con timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = os.path.join(AMPBConfig.OUTPUT_DIR, model_title)
    os.makedirs(log_dir, exist_ok=True)
    log_filename = os.path.join(log_dir, f"{model_title}_{timestamp}.log")
    
    # Lock para escribir al archivo de forma thread-safe
    log_lock = threading.Lock()
    
    # Función para escribir al log de forma segura
    def write_to_log(message):
        with log_lock:
            with open(log_filename, 'a', encoding='utf-8') as log_file:
                log_file.write(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
                log_file.flush()
    
    # Procesamiento especial para exog_set_id: generar todas las combinaciones posibles
    processed_param_grid = param_grid.copy()
    
    if 'exog_set_id' in processed_param_grid:
        exog_values = processed_param_grid['exog_set_id']
        
        # Validar que todos los valores están en el rango 1-6
        valid_values = {1, 2, 3, 4, 5, 6}
        invalid_values = [v for v in exog_values if v not in valid_values]
        
        if invalid_values:
            error_msg = f"{AMPBConfig.COLOR_INFO}ERROR: Los valores de exog_set_id deben estar entre 1 y 6.\nValores inválidos encontrados: {invalid_values}\nGrid Search abortado.{AMPBConfig.COLOR_RESET}"
            print(error_msg)
            # Guardar error en log
            with open(log_filename, 'w', encoding='utf-8') as log_file:
                log_file.write(f"GRID SEARCH ERROR - {timestamp}\n")
                log_file.write(f"Model: {model_title}\n")
                log_file.write(f"Error: {error_msg}\n")
            return
        
        if len(exog_values) > 1:
            # Generar todas las combinaciones posibles (individuales, pares, triples, etc.)
            all_combinations = []
            
            # Añadir valores individuales
            for value in exog_values:
                all_combinations.append(str(value))
            
            # Añadir combinaciones de 2, 3, 4... elementos
            for r in range(2, len(exog_values) + 1):
                for combo in itertools.combinations(exog_values, r):
                    # Unir los números como string: (1,2) -> "12"
                    combined_str = ''.join(map(str, combo))
                    all_combinations.append(combined_str)
            
            processed_param_grid['exog_set_id'] = all_combinations
    
    # Preparar las combinaciones de parámetros
    keys = processed_param_grid.keys()
    values = processed_param_grid.values()
    combinations = list(itertools.product(*values))
        
    total_combinations = len(combinations)
    
    # Crear/abrir archivo de log
    with open(log_filename, 'w', encoding='utf-8') as log_file:
        # Escribir cabecera del log
        log_file.write(f"GRID SEARCH LOG (SECUENCIAL) - {timestamp}\n")
        log_file.write(f"{'='*60}\n")
        log_file.write(f"Model: {model_title}\n")
        log_file.write(f"Worker Notebook: {worker_notebook}\n")
        log_file.write(f"Total Combinations: {total_combinations}\n")
        log_file.write(f"Debug Mode: {debug_mode}\n")
        log_file.write(f"{'='*60}\n\n")
        log_file.flush()
    
    print(f"{AMPBConfig.COLOR_HEADER}="*60)
    print(f"GRID SEARCH PARA: {model_title}")    
    print(f"Se ejecutarán un total de {total_combinations} (x2 SV+BT) combinaciones de parámetros.")
    print(f"Notebook a ejecutar: {worker_notebook}")
    print(f"Log file: {log_filename}")
    print("="*60)    
    if debug_mode:
        print(f"{AMPBConfig.COLOR_KEY}*** MODO DEBUG ACTIVADO - NO SE EJECUTARÁN LOS COMANDOS ***")
    print(f"{AMPBConfig.COLOR_RESET}")
    
    start_time = time.time()
    successful_runs = 0
    failed_runs = []
    skipped_runs = 0
    eta_minutes = 0

    # Log del inicio de ejecución
    write_to_log(f"INICIO DE EJECUCIÓN - {total_combinations} combinaciones")

    # Iterar sobre cada combinación
    for i, combo in enumerate(combinations):
        params = dict(zip(keys, combo))
        
        # Log del inicio de la combinación
        write_to_log(f"INICIANDO Combinación {i+1}/{total_combinations}: {params}")
        
        # Estimar tiempo restante
        elapsed = time.time() - start_time
        if i==0:
            avg_time_per_combo = 1
        else:
            avg_time_per_combo = elapsed / i
        eta_minutes = (total_combinations - i) * avg_time_per_combo / 60
        print(f"{AMPBConfig.COLOR_VALUE}------------------------------------------------------------------------------------------")
        print(f"  Ejecutando combinación {i+1}/{total_combinations} | ETA: {eta_minutes:.1f} min ")
        print(f"  Parámetros: {params}")
        print(f"  {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"------------------------------------------------------------------------------------------\n")

        # Verificar si los archivos de salida ya existen
        if checkOutputFiles(model_title, params):
            write_to_log(f"SALTADA Combinación {i+1} - Archivos ya existen")
            skipped_runs += 1
            continue

        # Construcción del comando
        command = [
            sys.executable,
            '-m', 'IPython', 
            '--TerminalIPythonApp.file_to_run', 
            worker_notebook,
            '--',      # Separador
            '--dummy'  # Argumento falso para sacrificar
        ]
        
        # Añadir los parámetros reales al comando
        for key, value in params.items():
            command.append(f'--{key}')
            command.append(str(value))

        # Log del comando
        write_to_log(f"COMANDO Combinación {i+1}: {' '.join(command)}")

        # Si está en modo debug, solo mostrar el comando sin ejecutar
        if debug_mode:
            print(f"  {AMPBConfig.COLOR_KEY}MODO DEBUG: Comando que se ejecutaría:{AMPBConfig.COLOR_RESET}")
            print(f"    {' '.join(command)}")
            print(f"  {AMPBConfig.COLOR_KEY}Ejecución simulada completada.{AMPBConfig.COLOR_RESET}\n")
            write_to_log(f"DEBUG Combinación {i+1} - Simulación completada")
            successful_runs += 1
        else:
            try:
                # Ejecutar el comando, capturando la salida para el log
                result = subprocess.run(
                    command, 
                    check=True,
                    text=True,
                    encoding='utf-8',
                    capture_output=True
                )
                
                # Log del output exitoso
                write_to_log(f"ÉXITO Combinación {i+1}")
                if result.stdout:
                    write_to_log(f"STDOUT Combinación {i+1}:\n{result.stdout}")
                if result.stderr:
                    write_to_log(f"STDERR Combinación {i+1}:\n{result.stderr}")
                
                print(f"  {AMPBConfig.COLOR_VALUE}Ejecución completada exitosamente.{AMPBConfig.COLOR_RESET}")
                successful_runs += 1

            except subprocess.CalledProcessError as e:
                # Log del error
                error_msg = f"ERROR en la ejecución de la combinación {params}"
                write_to_log(f"ERROR Combinación {i+1}: {str(e)}")
                if e.stderr:
                    write_to_log(f"ERROR STDERR Combinación {i+1}:\n{e.stderr}")
                if e.stdout:
                    write_to_log(f"ERROR STDOUT Combinación {i+1}:\n{e.stdout}")
                
                print(f"  {AMPBConfig.COLOR_INFO}{error_msg}{AMPBConfig.COLOR_RESET}")
                failed_runs.append(params)
                # Imprimir stderr puede dar pistas sobre el error del notebook
                if e.stderr:
                    print(f"    --- Salida de Error (stderr) ---\n{e.stderr}")
                if e.stdout:
                    print(f"    --- Salida Estándar (stdout) ---\n{e.stdout}")
            
    # Resumen final
    end_time = time.time()
    total_time = end_time - start_time
    
    # Crear resumen final
    final_summary = []
    final_summary.append("="*60)
    final_summary.append(f"GRID SEARCH COMPLETADO: {model_title}")    
    final_summary.append(f"  - Combinaciones totales: {total_combinations}")
    final_summary.append(f"  - Ejecuciones exitosas: {successful_runs}")
    final_summary.append(f"  - Ejecuciones saltadas (archivos existentes): {skipped_runs}")
    final_summary.append(f"  - Ejecuciones fallidas: {len(failed_runs)}")
    if failed_runs:
        final_summary.append("    - Configuraciones que fallaron:")
        for failed_param in failed_runs:
            final_summary.append(f"      {failed_param}")
    final_summary.append(f"  - Tiempo total: {total_time / 60:.2f} minutos ({total_time:.2f} segundos)")
    if skipped_runs > 0:
        final_summary.append(f"  - Tiempo ahorrado aproximado: {skipped_runs * (total_time / max(1, successful_runs)) / 60:.2f} minutos")
    final_summary.append("="*60)    
    if debug_mode:
        final_summary.append(" *** MODO DEBUG - NINGÚN COMANDO FUE EJECUTADO REALMENTE ***")
    
    # Mostrar resumen en consola
    print(f"{AMPBConfig.COLOR_HEADER}")
    for line in final_summary:
        print(line)
    print(f"{AMPBConfig.COLOR_RESET}")
    
    # Escribir resumen final al log
    write_to_log("RESUMEN FINAL:")
    for line in final_summary:
        write_to_log(line)
    
    print(f"{AMPBConfig.COLOR_HEADER}Log guardado en: {log_filename}{AMPBConfig.COLOR_RESET}")

In [ ]:
# Version que se ejecuta en paralelo dependiendo de los nucleos de CPU (mucho mas rapida!)
def runGridSearchP(model_title, worker_notebook, param_grid, debug_mode=False, max_workers=None):
    # Si no se especifica max_workers, usar el número de cores del sistema
    if max_workers is None:
        max_workers = os.cpu_count()
    
    # Crear directorio para el modelo y archivo de log con timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = os.path.join(AMPBConfig.OUTPUT_DIR, model_title)
    os.makedirs(log_dir, exist_ok=True)
    log_filename = os.path.join(log_dir, f"{model_title}_{timestamp}.log")
    
    # Procesamiento especial para exog_set_id: generar todas las combinaciones posibles
    processed_param_grid = param_grid.copy()
    
    if 'exog_set_id' in processed_param_grid:
        exog_values = processed_param_grid['exog_set_id']
        
        # Validar que todos los valores están en el rango 1-6
        valid_values = {1, 2, 3, 4, 5, 6}
        invalid_values = [v for v in exog_values if v not in valid_values]
        
        if invalid_values:
            error_msg = f"{AMPBConfig.COLOR_INFO}ERROR: Los valores de exog_set_id deben estar entre 1 y 6.\nValores inválidos encontrados: {invalid_values}\nGrid Search abortado.{AMPBConfig.COLOR_RESET}"
            print(error_msg)
            # Guardar error en log
            with open(log_filename, 'w', encoding='utf-8') as log_file:
                log_file.write(f"GRID SEARCH ERROR - {timestamp}\n")
                log_file.write(f"Model: {model_title}\n")
                log_file.write(f"Error: {error_msg}\n")
            return
        
        if len(exog_values) > 1:
            # Generar todas las combinaciones posibles (individuales, pares, triples, etc.)
            all_combinations = []
            
            # Añadir valores individuales
            for value in exog_values:
                all_combinations.append(str(value))
            
            # Añadir combinaciones de 2, 3, 4... elementos
            for r in range(2, len(exog_values) + 1):
                for combo in itertools.combinations(exog_values, r):
                    # Unir los números como string: (1,2) -> "12"
                    combined_str = ''.join(map(str, combo))
                    all_combinations.append(combined_str)
            
            processed_param_grid['exog_set_id'] = all_combinations
    
    # Preparar las combinaciones de parámetros
    keys = processed_param_grid.keys()
    values = processed_param_grid.values()
    combinations = list(itertools.product(*values))
        
    total_combinations = len(combinations)
    
    # Crear/abrir archivo de log
    with open(log_filename, 'w', encoding='utf-8') as log_file:
        # Escribir cabecera del log
        log_file.write(f"GRID SEARCH LOG - {timestamp}\n")
        log_file.write(f"{'='*60}\n")
        log_file.write(f"Model: {model_title}\n")
        log_file.write(f"Worker Notebook: {worker_notebook}\n")
        log_file.write(f"Total Combinations: {total_combinations}\n")
        log_file.write(f"Max Workers: {max_workers}\n")
        log_file.write(f"Debug Mode: {debug_mode}\n")
        log_file.write(f"{'='*60}\n\n")
        log_file.flush()
    
    print(f"{AMPBConfig.COLOR_HEADER}="*60)
    print(f"GRID SEARCH PARALELO PARA: {model_title}")    
    print(f"Se ejecutarán un total de {total_combinations} (x2 SV+BT) combinaciones de parámetros.")
    print(f"Notebook a ejecutar: {worker_notebook}")
    print(f"Ejecución en paralelo: {max_workers} workers simultáneos")
    print(f"Log file: {log_filename}")
    print("="*60)    
    if debug_mode:
        print(f"{AMPBConfig.COLOR_KEY}*** MODO DEBUG ACTIVADO - NO SE EJECUTARÁN LOS COMANDOS ***")
    print(f"{AMPBConfig.COLOR_RESET}")
    
    # Lock para escribir al archivo de forma thread-safe
    log_lock = threading.Lock()
    
    # Función para escribir al log de forma segura
    def write_to_log(message):
        with log_lock:
            with open(log_filename, 'a', encoding='utf-8') as log_file:
                log_file.write(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
                log_file.flush()
    
    # Función para ejecutar una combinación individual
    def execute_combination(combo_data):
        i, combo = combo_data
        params = dict(zip(keys, combo))
        
        # Log del inicio de la combinación
        write_to_log(f"INICIANDO Combinación {i+1}/{total_combinations}: {params}")
        
        # Verificar si los archivos de salida ya existen
        if checkOutputFiles(model_title, params):
            write_to_log(f"SALTADA Combinación {i+1} - Archivos ya existen")
            return {'success': True, 'params': params, 'combo_num': i+1, 'skipped': True}

        print(f"{AMPBConfig.COLOR_VALUE}Ejecutando combinación {i+1}/{total_combinations} - {params}")
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        # Construcción del comando
        command = [
            sys.executable,
            '-m', 'IPython', 
            '--TerminalIPythonApp.file_to_run', 
            worker_notebook,
            '--',      # Separador
            '--dummy'  # Argumento falso para sacrificar
        ]
        
        # Añadir los parámetros reales al comando
        for key, value in params.items():
            command.append(f'--{key}')
            command.append(str(value))

        # Log del comando
        write_to_log(f"COMANDO Combinación {i+1}: {' '.join(command)}")

        # Si está en modo debug, solo mostrar el comando sin ejecutar
        if debug_mode:
            print(f"  {AMPBConfig.COLOR_KEY}MODO DEBUG: Comando que se ejecutaría:{AMPBConfig.COLOR_RESET}")
            print(f"    {' '.join(command)}")
            print(f"  {AMPBConfig.COLOR_KEY}Ejecución simulada completada.{AMPBConfig.COLOR_RESET}")
            write_to_log(f"DEBUG Combinación {i+1} - Simulación completada")
            return {'success': True, 'params': params, 'combo_num': i+1, 'skipped': False}
        else:
            try:
                # Ejecutar el comando
                result = subprocess.run(
                    command, 
                    check=True,
                    text=True,
                    encoding='utf-8',
                    capture_output=True  # Capturar output para logging
                )
                
                # Log del output exitoso
                write_to_log(f"ÉXITO Combinación {i+1}")
                if result.stdout:
                    write_to_log(f"STDOUT Combinación {i+1}:\n{result.stdout}")
                if result.stderr:
                    write_to_log(f"STDERR Combinación {i+1}:\n{result.stderr}")
                
                print(f"  {AMPBConfig.COLOR_VALUE}Combinación {i+1} completada exitosamente.{AMPBConfig.COLOR_RESET}")
                return {'success': True, 'params': params, 'combo_num': i+1, 'skipped': False}

            except subprocess.CalledProcessError as e:
                # Log del error
                error_msg = f"ERROR en la ejecución de la combinación {i+1}: {params}"
                write_to_log(f"ERROR Combinación {i+1}: {str(e)}")
                if e.stderr:
                    write_to_log(f"ERROR STDERR Combinación {i+1}:\n{e.stderr}")
                if e.stdout:
                    write_to_log(f"ERROR STDOUT Combinación {i+1}:\n{e.stdout}")
                
                print(f"  {AMPBConfig.COLOR_INFO}{error_msg}{AMPBConfig.COLOR_RESET}")
                if e.stderr:
                    print(f"    --- Salida de Error (stderr) ---\n{e.stderr}")
                if e.stdout:
                    print(f"    --- Salida Estándar (stdout) ---\n{e.stdout}")
                return {'success': False, 'params': params, 'combo_num': i+1, 'error': str(e), 'skipped': False}

    start_time = time.time()
    successful_runs = 0
    failed_runs = []
    skipped_runs = 0
    
    # Log del inicio de ejecución
    write_to_log(f"INICIO DE EJECUCIÓN - {total_combinations} combinaciones")
    
    # Ejecutar en paralelo usando ThreadPoolExecutor
    if debug_mode:
        # En modo debug, ejecutar secuencialmente para mantener orden en output
        print(f"{AMPBConfig.COLOR_KEY}Modo debug: ejecutando secuencialmente para claridad del output{AMPBConfig.COLOR_RESET}\n")
        for i, combo in enumerate(combinations):
            result = execute_combination((i, combo))
            if result['success']:
                if result['skipped']:
                    skipped_runs += 1
                else:
                    successful_runs += 1
            else:
                failed_runs.append(result['params'])
    else:
        # Ejecución paralela real
        print(f"{AMPBConfig.COLOR_VALUE}Iniciando ejecución paralela con {max_workers} workers...{AMPBConfig.COLOR_RESET}\n")
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Enviar todas las tareas
            future_to_combo = {
                executor.submit(execute_combination, (i, combo)): (i, combo) 
                for i, combo in enumerate(combinations)
            }
            
            completed_count = 0
            # Procesar resultados conforme van completándose
            for future in as_completed(future_to_combo):
                completed_count += 1
                elapsed = time.time() - start_time
                eta_minutes = (total_combinations - completed_count) * (elapsed / completed_count) / 60
                
                active_workers = len([f for f in future_to_combo if not f.done()])
                progress_msg = f"Progress: {completed_count}/{total_combinations} | ETA: {eta_minutes:.1f} min | Activos: {active_workers}"
                print(f"{AMPBConfig.COLOR_KEY}{progress_msg}{AMPBConfig.COLOR_RESET}")
                write_to_log(f"PROGRESO: {progress_msg}")
                
                try:
                    result = future.result()
                    if result['success']:
                        if result['skipped']:
                            skipped_runs += 1
                        else:
                            successful_runs += 1
                    else:
                        failed_runs.append(result['params'])
                except Exception as exc:
                    combo_data = future_to_combo[future]
                    params = dict(zip(keys, combo_data[1]))
                    exc_msg = f'Combinación {combo_data[0]+1} generó excepción: {exc}'
                    print(f'  {AMPBConfig.COLOR_INFO}{exc_msg}{AMPBConfig.COLOR_RESET}')
                    write_to_log(f"EXCEPCIÓN: {exc_msg}")
                    failed_runs.append(params)
            
    # Resumen final
    end_time = time.time()
    total_time = end_time - start_time
    
    # Crear resumen final
    final_summary = []
    final_summary.append("="*60)
    final_summary.append(f"GRID SEARCH PARALELO COMPLETADO: {model_title}")    
    final_summary.append(f"  - Combinaciones totales: {total_combinations}")
    final_summary.append(f"  - Ejecuciones exitosas: {successful_runs}")
    final_summary.append(f"  - Ejecuciones saltadas (archivos existentes): {skipped_runs}")
    final_summary.append(f"  - Ejecuciones fallidas: {len(failed_runs)}")
    if failed_runs:
        final_summary.append("    - Configuraciones que fallaron:")
        for failed_param in failed_runs:
            final_summary.append(f"      {failed_param}")
    final_summary.append(f"  - Tiempo total: {total_time / 60:.2f} minutos ({total_time:.2f} segundos)")
    if skipped_runs > 0:
        executed_runs = successful_runs + len(failed_runs)
        if executed_runs > 0:
            avg_time_per_execution = total_time / executed_runs
            time_saved = skipped_runs * avg_time_per_execution / 60
            final_summary.append(f"  - Tiempo ahorrado aproximado: {time_saved:.2f} minutos")
    if not debug_mode and successful_runs > 0:
        theoretical_speedup = max_workers * (successful_runs + len(failed_runs)) / total_combinations
        final_summary.append(f"  - Speedup efectivo vs secuencial: ~{theoretical_speedup:.1f}x con {max_workers} workers")
    final_summary.append("="*60)    
    if debug_mode:
        final_summary.append(" *** MODO DEBUG - NINGÚN COMANDO FUE EJECUTADO REALMENTE ***")
    
    # Mostrar resumen en consola
    print(f"{AMPBConfig.COLOR_HEADER}")
    for line in final_summary:
        print(line)
    print(f"{AMPBConfig.COLOR_RESET}")
    
    # Escribir resumen final al log
    write_to_log("RESUMEN FINAL:")
    for line in final_summary:
        write_to_log(line)
    
    print(f"{AMPBConfig.COLOR_HEADER}Log guardado en: {log_filename}{AMPBConfig.COLOR_RESET}")

In [ ]:
def displayFinalResults(model_name, model_test, output_files=True, top_n_for_plot=15):
    import os, glob
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go

    test_title = f"Test: {model_test}"
    
    search_dir = os.path.join(AMPBConfig.OUTPUT_DIR, model_name, model_test)
    search_pattern = os.path.join(search_dir, f"{model_name}_{model_test}_*.csv")
    result_files = glob.glob(search_pattern)

    if not result_files:
        print(f"{AMPBConfig.COLOR_INFO}No se encontraron archivos de resultados para el patrón: '{search_pattern}'{AMPBConfig.COLOR_RESET}")
        return

    print(f"Se encontraron {len(result_files)} archivos de resultados para '{model_name} {model_test}'. Cargando...")

    try:
        all_results_df = pd.concat([pd.read_csv(f, sep=';', decimal=',') for f in result_files], ignore_index=True)
    except Exception as e:
        print(f"{AMPBConfig.COLOR_INFO}Error al cargar o concatenar los archivos CSV: {e}{AMPBConfig.COLOR_RESET}")
        return

    ranked_models = all_results_df.sort_values(by='irb_total', ascending=False).reset_index(drop=True)
    title = f"Ranking de modelos '{model_name}' - {test_title}"
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))

    display_cols_ordered = {
        'model_title': 'Configuración del modelo', 'model_hash': 'Hash', 'r2': 'R²', 'mae': 'MAE', 'rmse': 'RMSE',
        'accuracy': 'Acc.', 'f1_score': 'F1', 'roc_auc': 'ROC-AUC', 'irb_regr': 'IRB Regr.',
        'irb_clas': 'IRB Clas.', 'irb_total': 'IRB Total'
    }

    table_df = ranked_models[[col for col in display_cols_ordered if col in ranked_models.columns]].copy()
    table_df.rename(columns=display_cols_ordered, inplace=True)

    for col in ['IRB Total', 'IRB Regr.', 'IRB Clas.', 'R²', 'Acc.', 'F1', 'ROC-AUC']:
        if col in table_df.columns:
            table_df[col] = table_df[col].apply(lambda x: f"{x:.3f}")
    for col in ['MAE', 'RMSE']:
        if col in table_df.columns:
            table_df[col] = table_df[col].apply(lambda x: f"{x:.4f}")

    print(table_df.to_markdown(index=False))

    plot_data = ranked_models.head(top_n_for_plot)
    print(f"\nGenerando gráfico con los {len(plot_data)} mejores modelos...")

    fig = go.Figure()

    # En IRB, el baseline es 1
    fig.add_trace(go.Scatter(
        x=[1], y=[1],
        mode='markers',
        marker=dict(color='red', symbol='star', size=14, line=dict(color='yellow', width=2)),
        name='Predictor Base',
        hoverinfo='text',
        text=f"<b>Predictor Base</b><br>IRB Total: 1.0<br>IRB Regr: 1.0<br>IRB Clas: 1.0"
    ))

    scatter_fig = px.scatter(
        plot_data, x='irb_regr', y='irb_clas', color='model_title',
        color_discrete_sequence=px.colors.qualitative.Plotly,
        hover_name='model_title',
        hover_data={'model_title': False, 'irb_regr': ':.3f', 'irb_clas': ':.3f', 'irb_total': ':.3f'}
    )

    scatter_fig.update_traces(marker=dict(size=12))

    for trace in scatter_fig.data:
        fig.add_trace(trace)

    # Calcular los máximos y mínimos con margen
    x_max = plot_data['irb_regr'].max()
    x_min = plot_data['irb_regr'].min()
    y_max = plot_data['irb_clas'].max()
    y_min = plot_data['irb_clas'].min()
    # Aplicar márgenes del 5% y asegurar al menos 1.1 como máximo superior
    x_lim_max = round(max(x_max * 1.05, 1.1), 3)
    y_lim_max = round(max(y_max * 1.05, 1.1), 3)
    # Aplicar márgenes inferiores
    x_lim_min = round(min(x_min * 0.95, -0.05), 3)
    y_lim_min = round(min(y_min * 0.95, -0.05), 3)

    fig.update_layout(
        title_text=f"Rendimiento Regresión vs. Clasificación (Top {len(plot_data)})<br>Índice Relativo al Baseline (IRB)<br><sup>{title}</sup>",
        title_x=0.5,
        title_font=dict(size=20, family="Arial", color="black"),
        xaxis_title="Índice Relativo al Baseline (IRB) de Regresión (mejor →)",
        yaxis_title="Índice Relativo al Baseline (IRB) de Clasificación (mejor →)",
        xaxis=dict(
            range=[x_lim_min, x_lim_max],
            showgrid=True,
            gridcolor='LightGray',
            minor=dict(showgrid=True, gridcolor='LightGray', griddash='dot')
        ),
        yaxis=dict(
            range=[y_lim_min, y_lim_max],
            showgrid=True,
            gridcolor='LightGray',
            minor=dict(showgrid=True, gridcolor='LightGray', griddash='dot')
        ),
        width=1200, height=1000,
        plot_bgcolor='white',
        legend_title_text="",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5,
            traceorder='normal'
        )
    )

    mean_regr = plot_data['irb_regr'].mean()
    mean_clas = plot_data['irb_clas'].mean()
    # Línea vertical en IRB Regr.
    fig.add_shape(
        type="line",
        x0=mean_regr, y0=y_lim_min,
        x1=mean_regr, y1=y_lim_max,
        line=dict(color="DimGray", width=2, dash="dash")
    )
    fig.add_annotation(
        x=mean_regr,
        y=y_lim_min + 0.025 * (y_lim_max - y_lim_min),
        text=f"Media IRB Regr. ({mean_regr:.2f})",
        showarrow=False,
        font=dict(color="DimGray"),
        yanchor="bottom",
        xanchor="center"
    )
    # Línea horizontal en IRB Clas.
    fig.add_shape(
        type="line",
        x0=x_lim_min, y0=mean_clas,
        x1=x_lim_max, y1=mean_clas,
        line=dict(color="DimGray", width=2, dash="dash")
    )
    fig.add_annotation(
        x=x_lim_min + 0.02 * (x_lim_max - x_lim_min),
        y=mean_clas - 0.025 * (y_lim_max - y_lim_min),
        text=f"Media IRB Clas. ({mean_clas:.2f})",
        showarrow=False,
        font=dict(color="DimGray"),
        xanchor='left'
    )
    
    fig.show()

    if output_files:
        try:
            output_prefix = f"{model_name}_{model_test}_FinalResult"
            output_dir = AMPBConfig.OUTPUT_DIR

            image_filename = os.path.join(output_dir, f"{output_prefix}.png")
            fig.write_image(image_filename, scale=2)
            print(f"\n{AMPBConfig.COLOR_MSG}Gráfico de comparación guardado como: '{image_filename}'{AMPBConfig.COLOR_RESET}")

            csv_filename = os.path.join(output_dir, f"{output_prefix}.csv")
            cols_to_save = [col for col in display_cols_ordered.keys() if col in ranked_models.columns]
            full_table_df = ranked_models[cols_to_save].copy()
            full_table_df.rename(columns=display_cols_ordered, inplace=True)
            full_table_df.to_csv(csv_filename, sep=';', decimal=',', index=False, encoding='utf-8-sig')
            print(f"{AMPBConfig.COLOR_MSG}Tabla completa de resultados guardada como: '{csv_filename}'{AMPBConfig.COLOR_RESET}")

        except Exception as e:
            print(f"{AMPBConfig.COLOR_INFO}ERROR al guardar los archivos de salida: {e}{AMPBConfig.COLOR_RESET}")


In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS ARIMA

assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

arima_param_grid = {
    'transformation': ['None', 'Log', 'RetLog', 'YeoJohnson']    
}

runGridSearch(
    model_title="ARIMA",
    worker_notebook="Predictor ARIMA.ipynb" ,
    param_grid=arima_param_grid
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE ARIMA
displayFinalResults(model_name="ARIMA",  model_test="SV")

displayFinalResults(model_name="ARIMA",  model_test="BT")
# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS PROPHET

assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

prophet_param_grid = {
    'transformation': ['None', 'Log', 'RetLog', 'YeoJohnson']    
}

runGridSearch(
    model_title="Prophet",
    worker_notebook="Predictor Prophet.ipynb" ,
    param_grid=prophet_param_grid
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE PROPHET
displayFinalResults(model_name="Prophet",  model_test="SV")

displayFinalResults(model_name="Prophet",  model_test="BT")
# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS SARIMAX

#assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

sarimax_param_grid = {
    #'transformation': ['YeoJohnson'],
    #'exog_scaling': ['None'],
    #'exog_set_id': [1,2,3]
    'transformation': ['RetLog'], #'YeoJohnson', 'None', 'Log',
    'exog_scaling': ['None', 'Standard', 'MinMax'],
    'exog_set_id': [1, 2, 3, 4, 5, 6]
}

runGridSearchP(
    model_title="SARIMAX",
    worker_notebook="Predictor SARIMAX.ipynb" ,
    param_grid=sarimax_param_grid,
    debug_mode=False,
    max_workers=12
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE SARIMAX
displayFinalResults(model_name="SARIMAX",  model_test="SV")

displayFinalResults(model_name="SARIMAX",  model_test="BT")
# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS RANDOM FOREST

assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

randomforest_param_grid = {    
    'exog_set_id': [1, 2, 3, 4, 5, 6]
}

runGridSearchP(
    model_title="RandomForest",
    worker_notebook="Predictor Random Forest.ipynb" ,
    param_grid=randomforest_param_grid,
    debug_mode=False,
    max_workers=8
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE RANDOM FOREST
displayFinalResults(model_name="RandomForest",  model_test="SV")

displayFinalResults(model_name="RandomForest",  model_test="BT")
# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS XGBOOST

assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

xgboost_param_grid = {    
    'exog_set_id': [1, 2, 3, 4, 5, 6]
}

runGridSearch(
    model_title="XGBoost",
    worker_notebook="Predictor XGBoost.ipynb" ,
    param_grid=xgboost_param_grid,
    debug_mode=False,
    #max_workers=8
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE XGBOOST
displayFinalResults(model_name="XGBoost",  model_test="SV")

displayFinalResults(model_name="XGBoost",  model_test="BT")
# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS LSTM

assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

lstm_param_grid = {    
    'transformation': ['Log', 'YeoJohnson'],
    'exog_scaling': ['Standard'],
    'exog_set_id': [1, 2, 3, 4, 5, 6]
}

runGridSearch(
    model_title="LSTM",
    worker_notebook="Predictor LSTM.ipynb" ,
    param_grid=lstm_param_grid,
    debug_mode=False,
    #max_workers=4
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE LSTM
displayFinalResults(model_name="LSTM",  model_test="SV")

displayFinalResults(model_name="LSTM",  model_test="BT")
# ----------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# EJECUCIÓN DE TODOS LOS MODELOS TRANSFORMER

#assert False, "¡Comenta esta línea si deseas realizar una evaluación completa de los modelos!"

lstm_param_grid = {    
    'transformation': ['Log', 'YeoJohnson'],
    'exog_scaling': ['Standard'],
    'exog_set_id': [1, 2, 3, 4, 5, 6]
}

runGridSearch(
    model_title="Transformer",
    worker_notebook="Predictor Transformer.ipynb" ,
    param_grid=lstm_param_grid,
    debug_mode=False,
    #max_workers=4
)

In [ ]:
# MOSTRAR RESULTADOS FINALES DE TRANSFORMER
displayFinalResults(model_name="Transformer",  model_test="SV")

displayFinalResults(model_name="Transformer",  model_test="BT")
# ----------------------------------------------------------------------